<a href="https://colab.research.google.com/github/0jipy/192kbps_colab_JIPY/blob/main/%5B%EC%BD%94%EB%93%9C%5D_%EB%8D%B0%EC%9D%B4%EC%BD%98_%EA%B8%B0%EC%B4%88_%EB%B2%A0%EC%9D%B4%EC%8A%A4%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **데이터 살펴보기**

In [4]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [2]:
#경로 설정
import os
# os.chdir('/content/drive/My Drive/월간 09/Writing_Style/Writing_Style')
os.chdir('/content/drive/MyDrive/DEEPLearning/data/235670_소설 작가 분류 AI 경진대회_data')

In [5]:
#파일 불러오기
train = pd.read_csv('open/train.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [ ]:
#train 데이터 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [6]:
#test 데이터 살펴보기
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [ ]:
#sample_submission
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


# **전처리**

In [9]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [10]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [11]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [12]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [13]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])


In [14]:
X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane not written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere wellwisher friend sister lucy odin',
       'wanted lend money', 'certainly not occurred said yes like'],
      dtype='<U1433')

# **모델링**

In [15]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [16]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [17]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [18]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [20]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 16)           320000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 5)                 125       
                                                                 
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 9s - loss: 1.5572 - accuracy: 0.2821 - val_loss: 1.5070 - val_accuracy: 0.2981 - 9s/epoch - 6ms/step
Epoch 2/20
1372/1372 - 8s - loss: 1.3780 - accuracy: 0.4084 - val_loss: 1.2725 - val_accuracy: 0.4585 - 8s/epoch - 6ms/step
Epoch 3/20
1372/1372 - 8s - loss: 1.1861 - accuracy: 0.5002 - val_loss: 1.1552 - val_accuracy: 0.5117 - 8s/epoch - 6ms/step
Epoch 4/20
1372/1372 - 8s - loss: 1.0801 - accuracy: 0.5567 - val_loss: 1.0936 - val_accuracy: 0.5475 - 8s/epoch - 6ms/step
Epoch 5/20
1372/1372 - 8s - loss: 1.0004 - accuracy: 0.6021 - val_loss: 1.0380 - val_accuracy: 0.5821 - 8s/epoch - 6ms/step
Epoch 6/20
1372/1372 - 8s - loss: 0.9256 - accuracy: 0.6440 - val_loss: 0.9756 - val_accuracy: 0.6176 - 8s/epoch - 6ms/step
Epoch 7/20
1372/1372 - 8s - loss: 0.8548 - accuracy: 0.6793 - val_loss: 0.9222 - val_accuracy: 0.6488 - 8s/epoch - 6ms/step
Epoch 8/20
1372/1372 - 8s - loss: 0.7934 - accuracy: 0.7087 - val_loss: 0.8799 - val_accuracy: 0.6656 - 8s/epoch - 6ms/step
Epoch 9/

In [23]:
# predict values
pred = model.predict(test_padded)

In [24]:
pred

array([[6.1301836e-03, 3.4134942e-01, 7.1165219e-02, 5.2209276e-01,
        5.9262391e-02],
       [1.1876368e-01, 6.6799426e-01, 2.1387056e-02, 2.1176042e-02,
        1.7067884e-01],
       [9.9447989e-01, 4.6806326e-03, 3.1583561e-06, 9.2596252e-08,
        8.3615235e-04],
       ...,
       [1.1311003e-03, 9.9884015e-01, 2.6482883e-09, 2.6141597e-05,
        2.5869742e-06],
       [6.9419906e-04, 9.9919683e-01, 8.9197158e-08, 3.7422345e-05,
        7.1418086e-05],
       [9.9939919e-01, 2.4130270e-04, 5.4933280e-05, 1.1001349e-06,
        3.0359600e-04]], dtype=float32)

In [ ]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,0.000002,0.788913,2.079758e-01,2.933731e-03,1.754381e-04
1,1,0.039714,0.568273,2.801114e-01,9.527192e-03,1.023743e-01
2,2,0.986493,0.002893,7.934828e-06,3.005905e-08,1.060580e-02
3,3,0.000009,0.000008,5.932612e-01,1.277321e-04,4.065941e-01
4,4,0.718228,0.002316,5.026389e-03,2.701174e-01,4.311929e-03
...,...,...,...,...,...,...
19612,19612,0.000003,0.999997,8.058678e-11,2.074077e-15,1.258322e-09
19613,19613,0.000435,0.000004,4.560322e-06,1.804264e-14,9.995571e-01
19614,19614,0.000043,0.999747,1.498263e-04,6.182356e-08,6.037653e-05
19615,19615,0.000030,0.999638,2.880485e-04,3.136330e-07,4.381185e-05


In [25]:
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')